In [1]:
import streamlit as st
from streamlit_timeline import timeline
import json
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
from ydata_profiling import ProfileReport
from streamlit_pandas_profiling import st_profile_report
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv('stroke.csv', index_col=0)
display(df)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0
...,...,...,...,...,...,...,...,...,...,...,...
15299,Female,22.0,0,0,No,Govt_job,Urban,72.63,19.5,never smoked,0
15300,Female,46.0,1,0,Yes,Private,Urban,101.19,32.1,never smoked,0
15301,Female,75.0,0,0,Yes,Self-employed,Urban,87.69,26.2,never smoked,0


In [8]:
# train and test split
X = df.drop("stroke", axis=1)
y = df.stroke
split = train_test_split(X, y, test_size = 0.2, shuffle=True, random_state=4)

In [35]:
def train_model(model, train_test_split, metric):    
    X_train, X_test, y_train, y_test = train_test_split
    folds = 5
    
    param_grid = {
        'knn__n_neighbors': [3, 5, 7, 9],  # Number of neighbors
        'knn__weights': ['uniform', 'distance'],  # Weight function used in prediction
        'knn__metric': ['euclidean', 'manhattan', 'minkowski']  # Distance metric
    }


    # Create a GridSearchCV object with cross-validation
    grid_search = GridSearchCV(model, param_grid, cv=folds, n_jobs=-1, scoring=metric, refit=True, return_train_score=True)

    # Fit the model on the training data
    grid_search.fit(X_train, y_train)

    
    # Evaluate the model on the test set
    test_score = grid_search.score(X_test, y_test)
    
    return grid_search.best_estimator_, grid_search.best_params_, grid_search.best_score_, test_score

In [33]:
def ML_Pipeline(df,features,target,train_test_split, models, metric):
    X = df[features]
    y = df[target]      

    stats = np.zeros(shape=(2,len(models)))
    hyperparams = []
    trained_models = []

    for i,model in enumerate(models):
        trained_model, best_params, val_score, test_score = train_model(model, train_test_split, metric)
        stats[0,i] = val_score
        stats[1,i] = test_score
        trained_models.append(trained_model)
    return stats, trained_models,hyperparams

def get_best_model(ML_task,metric,models,reg_models,clas_models):
    if ML_task == "Classification":
        index = np.argmax(metric[1,:])
        best_value = np.max(metric[1,:])
        model= clas_models[index]
    else:
        index = np.argmin(metric[1,:])
        best_value = np.min(metric[1,:])
        model= clas_models[index]

    return model,best_value,index

In [38]:
ML_Pipeline(df, X.columns, "stroke", split, [LogisticRegression()], "accuracy")

ValueError: Invalid parameter 'knn' for estimator LogisticRegression(). Valid parameters are: ['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'].